In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import nltk.stem
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
from string import punctuation
from multiprocessing import Pool
from tqdm import tqdm

tqdm.pandas()

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/infres/bouharra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/infres/bouharra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
stop_words = set(stopwords.words('english'))

stop_words.add("'s")
stop_words.add("'")
stop_words.add("-")
stop_words.add("'")

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.head(2)

,title,description,channel,publishTime,tags,category,duration,commentNumber,likesNumber,views,...,quality,license,videoId,trendingTimes,trendingDuration,rawText,clean,wordCount,transDes,transText
0,Aya Nakamura - Dégaine feat. Damso (Clip offic...,Nouveau single 'Dégaine' en featuring avec Dam...,Aya Nakamura,2022-03-09 16:00:02+00:00,[],10,3:29,8548,330388,28553344,...,hd,1.0,7Lp9clJSB7E,8.0,7 days 00:00:00,"J""ai vu comment tu m'as regardée mon charme a...","J""ai vu comment tu m'as regardée mon charme fa...",451,New single 'Dégaine' featuring Damso available...,I saw how you looked at me my charm had its ef...
1,Aya Nakamura - Bobo (Clip officiel),#tiktoksong #chilikuku #hmmhmm\n\nNouveau sing...,Aya Nakamura,2021-05-27 15:00:12+00:00,[],10,3:14,13982,521150,50240770,...,hd,1.0,3zsPWw2H9PE,11.0,5 days 00:00:00,Chéri Coco fais-moi mmm mmm J’veux le bifton p...,Chéri Coco fais-moi mmm mmm J’veux le bifton p...,380,#tiktoksong #chilikuku #hmmhmm New single 'Bob...,Chéri Coco give me mmm mmm I want the bifton n...


In [5]:
df['transDes'] = df['transDes'].str.lower().map(lambda x: re.sub('\W+',' ', x))
df['transText'] = df['transText'].str.lower().map(lambda x: re.sub('\W+',' ', x))

In [6]:
def tokenize(i):
    return word_tokenize(df.transText[i]),word_tokenize(df.transDes[i]),i

def noStop(x):
    return [w for w in x if not w in stop_words and not w in punctuation]

In [7]:
tokenText = ['' for _ in range(len(df))]
tokenDes = ['' for _ in range(len(df))]

In [8]:
stemText = ['' for _ in range(len(df))]
stemDes = ['' for _ in range(len(df))]

In [9]:
pool = Pool(processes=80)
for t,d,i in tqdm(pool.imap(tokenize, df.index.tolist()), total=len(df)):
    tokenDes[i] = d
    tokenText[i] = t

df['tokenDes'] = tokenDes
df['tokenText'] = tokenText

100%|███████████████████████████████████| 11552/11552 [00:10<00:00, 1132.60it/s]


In [10]:
df['cleanText'] = df.tokenText.apply(noStop)
df['cleanDes'] = df.tokenDes.apply(noStop)

In [11]:
stemmer = nltk.stem.PorterStemmer()

def stemming(speech):
    return [format(stemmer.stem(token)) for token in speech]

In [12]:
df['stemsText'] = df['cleanText'].progress_apply(stemming)
df['stemsDes'] = df['cleanDes'].progress_apply(stemming)

100%|████████████████████████████████████| 11552/11552 [01:21<00:00, 141.02it/s]


In [14]:
df.to_pickle('stemmed.pkl')